# Intsall Package

In [2]:
# Install relevant packages, if not yet
!pip3 install awswrangler
!pip3 install pandasql

  Using cached awswrangler-2.16.1-py3-none-any.whl (248 kB)
  Using cached progressbar2-4.0.0-py2.py3-none-any.whl (26 kB)
  Using cached redshift_connector-2.0.908-py3-none-any.whl (112 kB)
  Using cached jsonpath_ng-1.5.3-py3-none-any.whl (29 kB)
  Using cached pg8000-1.29.1-py3-none-any.whl (34 kB)
  Using cached backoff-2.1.2-py3-none-any.whl (14 kB)
  Using cached requests_aws4auth-1.1.2-py2.py3-none-any.whl (24 kB)
  Using cached opensearch_py-1.1.0-py2.py3-none-any.whl (207 kB)
  Using cached pyarrow-7.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Using cached gremlinpython-3.6.0-py2.py3-none-any.whl (72 kB)
  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached aenum-3.1.11-py3-none-any.whl (131 kB)
  Using cached scramp-1.4.1-py3-none-any.whl (8.5 kB)
  Using cached python_utils-3.3.3

In [1]:
from data_connector import *
import numpy as np
import pandas as pd

# Import Packages and Connect to RedShift

In [5]:
# Fill in the relevant credentials for Redshift cluster(s) and S3 bucket(s) to use
role_arn = 'arn:aws:iam::848271137894:role/RedShiftAccessFromSageMakerAccount'
role_session_name = 'AssumeRoleSession1'
secret_name = 'arn:aws:secretsmanager:us-west-2:848271137894:secret:Redshift-DLL-Sagemaker-Secret-nHWV37'
dbname = 'ucicanvas'
region_name = 'us-west-2'
bucket_name = 'uci-dll-mellon-redshift-unload-us-west-2'
unload_iam_role = 'arn:aws:iam::848271137894:role/uci-write-to-dll-mellon-redshift-unload-s3-bucket-role,arn:aws:iam::686457044982:role/uci-oit-dwh-redshift-to-mellon-s3-unload-role'

In [6]:
# Initiate a connector object
redshift_s3_connect = RedshiftS3Connector(role_arn, role_session_name, secret_name, dbname, bucket_name, region_name, unload_iam_role)

# Get the discussion_post layer2 table from Fall 2020 to Spring 2022

In [11]:
get_df_query = f'''SELECT dp.* 
FROM mellon_layer2.v_discussion_post AS dp INNER JOIN 
(SELECT DISTINCT mellon_id, canvas_course_id
FROM mellon_layer2.enrollment
WHERE is_valid_user AND is_valid_course_section AND enrollment_type = \\'StudentEnrollment\\') AS en
ON dp.mellon_id = en.mellon_id AND dp.canvas_course_id = en.canvas_course_id
WHERE dp.discussion_post_created_at >= \\'2020-10-01 00:00:00\\' AND dp.discussion_post_created_at <= \\'2022-06-10 00:00:00\\';'''
print(get_df_query)

SELECT dp.* 
FROM mellon_layer2.v_discussion_post AS dp INNER JOIN 
(SELECT DISTINCT mellon_id, canvas_course_id
FROM mellon_layer2.enrollment
WHERE is_valid_user AND is_valid_course_section AND enrollment_type = \'StudentEnrollment\') AS en
ON dp.mellon_id = en.mellon_id AND dp.canvas_course_id = en.canvas_course_id
WHERE dp.discussion_post_created_at >= \'2020-10-01 00:00:00\' AND dp.discussion_post_created_at <= \'2022-06-10 00:00:00\';


In [12]:
file_in_s3 = 'discussion_post_2020_2022.csv'

In [13]:
redshift_s3_connect.unload_from_redshift_to_s3(get_df_query, file_in_s3, 'csv')

Done waiting to finish Data API.
Status: FINISHED. Excution time: 23032 miliseconds
{'ClusterIdentifier': 'uci-datalake-canvas-cluster-v1-redshiftcluster-a9h29wndif8j', 'CreatedAt': datetime.datetime(2022, 6, 18, 4, 38, 30, 675000, tzinfo=tzlocal()), 'Duration': 23032383315, 'HasResultSet': False, 'Id': '4284b2d1-0694-4b8e-9e60-09a2f4666f75', 'QueryString': "UNLOAD ('SELECT dp.* \nFROM mellon_layer2.v_discussion_post AS dp INNER JOIN \n(SELECT DISTINCT mellon_id, canvas_course_id\nFROM mellon_layer2.enrollment\nWHERE is_valid_user AND is_valid_course_section AND enrollment_type = \\'StudentEnrollment\\') AS en\nON dp.mellon_id = en.mellon_id AND dp.canvas_course_id = en.canvas_course_id\nWHERE dp.discussion_post_created_at >= \\'2020-10-01 00:00:00\\' AND dp.discussion_post_created_at <= \\'2022-06-10 00:00:00\\'') TO 's3://uci-dll-mellon-redshift-unload-us-west-2/discussion_post_2020_2022.csv' IAM_ROLE '' ALLOWOVERWRITE HEADER ENCRYPTED AUTO FORMAT CSV;", 'RedshiftPid': 1090455275, 'R

In [15]:
file_names = [file_in_s3]
file_type = 'csv'
discussion_post = redshift_s3_connect.read_from_s3(file_names, file_type)
discussion_post.head(5)

,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,parent_discussion_post_id,discussion_topic_id,discussion_topic_title,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id
0,2993273,186451,30814,"<p>Hello Chengzhi, I highly agree with your pe...",2020-10-17 17:10:10.051,2020-10-17 17:10:10.051,NaN,2,2962860.0,420159,Week 2: Discussion Board Assignment Section 7,NaN,2020-10-12 19:02:15.729,2020-10-12 19:02:16.07,NaN,f,NaN,609820.0
1,2994997,190837,29200,"<p><span style=""font-weight: 400;"">In this wee...",2020-10-18 00:06:37.544,2020-10-18 00:06:37.544,NaN,1,NaN,405368,Week 2 Discussion - Diversity,NaN,2020-09-29 15:37:13.63,2020-09-29 15:37:13.63,NaN,f,98110.0,551803.0
2,2995215,196032,30992,"<p>Hey Aaron,</p>\n<p>I see that we had simila...",2020-10-18 00:53:40.889,2020-10-18 00:53:40.889,NaN,3,2989109.0,411487,Discussion 2: Group Critique: Sample Elevator ...,NaN,2020-10-02 16:43:43.622,2020-10-02 16:43:43.623,NaN,f,100369.0,603402.0
3,3032164,183043,30112,<p>Given that this week was based very heavily...,2020-10-22 19:46:45.937,2020-10-22 19:47:33.675,NaN,1,NaN,399050,Discussion board: Sexual Revolutions: Hardcore...,NaN,2020-09-21 23:46:40.447,2020-10-16 07:00:02.067,2020-10-16 07:00:00,f,NaN,NaN
4,3102411,173790,30118,"<p>Hey Kirra,</p>\n<p>I agree with your perspe...",2020-11-01 04:01:13.621,2020-11-01 04:01:13.621,NaN,2,3087766.0,395262,Psych Fundamentals C Week 4 Message Board - Q...,NaN,2020-09-12 02:42:24.779,NaN,NaN,f,NaN,574673.0


In [6]:
df = pd.read_csv('discussion_post_2020_2022.csv')

In [8]:
print(df['discussion_post_content'][0])

<p>Hello Chengzhi, I highly agree with your perspective on how conceptual analysis can be fruitful. I especially enjoyed your example between humans and primates because our studies of primates we have grown a better understanding of or primal parts of human behavior. And as you stated by using our closest genetic relative we've been able to see some aspects of our society in their hierarchies.</p>


In [7]:
df_small = df.sample(n=100000)

In [8]:
df_small.to_csv('discussion_post_2020_2022_small.csv')

# Clean the data

In [2]:
import online_discourse_analysis as oda
import numpy as np
import pandas as pd

In [3]:
df_small = pd.read_csv('discussion_post_2020_2022_small.csv')

In [3]:
df_small = oda.discussion_text_clean(df_small)

In [5]:
df_small.head(5)

,Unnamed: 0,Unnamed: 0.1,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,...,discussion_topic_id,discussion_topic_title,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,cleaned_discussion_post
0,0,0,2993273,186451,30814,"<p>Hello Chengzhi, I highly agree with your pe...",2020-10-17 17:10:10.051,2020-10-17 17:10:10.051,NaN,2,...,420159,Week 2: Discussion Board Assignment Section 7,NaN,2020-10-12 19:02:15.729,2020-10-12 19:02:16.07,NaN,f,NaN,609820.0,"Hello Chengzhi, I highly agree with your persp..."
1,1,1,2994997,190837,29200,"<p><span style=""font-weight: 400;"">In this wee...",2020-10-18 00:06:37.544,2020-10-18 00:06:37.544,NaN,1,...,405368,Week 2 Discussion - Diversity,NaN,2020-09-29 15:37:13.63,2020-09-29 15:37:13.63,NaN,f,98110.0,551803.0,In this week’s first video lecture by Dacher K...
2,2,2,2995215,196032,30992,"<p>Hey Aaron,</p>\n<p>I see that we had simila...",2020-10-18 00:53:40.889,2020-10-18 00:53:40.889,NaN,3,...,411487,Discussion 2: Group Critique: Sample Elevator ...,NaN,2020-10-02 16:43:43.622,2020-10-02 16:43:43.623,NaN,f,100369.0,603402.0,"Hey Aaron,I see that we had similar ratings fo..."
3,3,3,3032164,183043,30112,<p>Given that this week was based very heavily...,2020-10-22 19:46:45.937,2020-10-22 19:47:33.675,NaN,1,...,399050,Discussion board: Sexual Revolutions: Hardcore...,NaN,2020-09-21 23:46:40.447,2020-10-16 07:00:02.067,2020-10-16 07:00:00,f,NaN,NaN,Given that this week was based very heavily on...
4,4,4,3102411,173790,30118,"<p>Hey Kirra,</p>\n<p>I agree with your perspe...",2020-11-01 04:01:13.621,2020-11-01 04:01:13.621,NaN,2,...,395262,Psych Fundamentals C Week 4 Message Board - Q...,NaN,2020-09-12 02:42:24.779,NaN,NaN,f,NaN,574673.0,"Hey Kirra,I agree with your perspective on dif..."


# Create term/quarter column

In [14]:
df = oda.create_term_col(df_small)

In [6]:
len(df[df['term'] == '0'])

690

In [10]:
df.head(5)

,Unnamed: 0,Unnamed: 0.1,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,...,discussion_topic_id,discussion_topic_title,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,term
0,818696,818696,3710696,184024,33621,<p>Going through the Kriger et. al reading sol...,2021-02-22 02:21:08.093000+00:00,2021-02-22 02:21:08.093,NaN,1,...,465175,Week 7 RPR,NaN,2020-12-28 17:13:43.494,2021-02-15 04:25:16.831000+00:00,NaN,f,NaN,654138.0,Winter 2021
1,1163402,1163402,3280790,189732,29719,<p>The drug I am looking for is loratadine. Th...,2020-11-30 06:14:56.718000+00:00,2020-11-30 06:14:56.718,NaN,1,...,408802,Week 8 Discussion 1 (Side-effects) - Group 27,NaN,2020-10-01 01:43:16.391,2020-10-01 01:43:16.391000+00:00,NaN,f,98950.0,575833.0,Fall 2020
2,1069337,1069337,3937321,180683,35411,<p>I totally agree with the idea that stress w...,2021-04-12 00:23:58.330000+00:00,2021-04-12 00:23:58.33,NaN,1,...,521980,"Discussion 2: The Etiology, Physiology, Sympto...",NaN,2021-03-20 00:55:55.57,2021-03-20 00:55:55.570000+00:00,NaN,f,122281.0,709312.0,Spring 2021
3,279733,279733,3419760,173435,34214,<p>Hello everyone!! My name is Darshit Shethna...,2021-01-09 10:59:23.040000+00:00,2021-01-09 10:59:54.513,NaN,1,...,478022,INTRODUCTIONS,NaN,2021-01-06 07:09:05.054,2021-01-06 07:09:05.469000+00:00,NaN,f,NaN,675548.0,Winter 2021
4,1055184,1055184,3812426,196511,34162,<p><span>Hypoactive Sexual Desire Disorder (AK...,2021-03-11 06:52:05.860000+00:00,2021-03-11 06:52:05.86,NaN,1,...,512474,Discussion 10: Health & Medicine - Group 4,NaN,2021-03-08 03:16:04.695,2021-03-08 03:16:04.698000+00:00,NaN,f,110753.0,702805.0,Winter 2021


# Calculate Behavioral Features

In [15]:
df = oda.discussion_topic_post_time_gap(df)

In [17]:
df = oda.calculate_features_avg(df, ['discussion_topic_post_time_gap'])

In [8]:
df = oda.discussion_post_count(df)

In [18]:
df.head(10)

,term,discussion_topic_post_time_gap_avg
0,0,30.423914
1,Fall 2020,21.495316
2,Fall 2021,27.240580
3,Spring 2021,19.848608
4,Spring 2022,27.877707
5,Summer 2021,17.694850
6,Winter 2021,17.389650
7,Winter 2022,19.990592


# Calculate Reply Rates

In [3]:
df = pd.read_csv('discussion_post_2020_2022_small.csv')

In [4]:
df = oda.discussion_reply_rate(df)

In [5]:
df.head(10)

,parent_discussion_post_id,reply_rate
0,2856555.0,4
1,2856833.0,1
2,2857538.0,1
3,2857868.0,1
4,2857922.0,1
5,2857943.0,1
6,2858074.0,1
7,2858104.0,1
8,2858181.0,1
9,2858194.0,1


# Remove running query

In [14]:
redshift_s3_connect.view_running_sql_commands()

Done waiting to finish Data API.
Status: FINISHED. Excution time: 59 miliseconds
{'ClusterIdentifier': 'uci-datalake-canvas-cluster-v1-redshiftcluster-a9h29wndif8j', 'CreatedAt': datetime.datetime(2022, 6, 18, 4, 39, 52, 544000, tzinfo=tzlocal()), 'Duration': 59145986, 'HasResultSet': True, 'Id': 'ae138db4-aced-45b0-b6b7-ade5b13f7e44', 'QueryString': "SELECT pid, TRIM(user_name), starttime, SUBSTRING(query, 1, 50)\n                         FROM stv_recents \n                         WHERE status='Running';\n                      ", 'RedshiftPid': 1090381566, 'RedshiftQueryId': 3504383, 'ResultRows': 0, 'ResultSize': 0, 'SecretArn': 'arn:aws:secretsmanager:us-west-2:848271137894:secret:Redshift-DLL-Sagemaker-Secret-nHWV37', 'Status': 'FINISHED', 'UpdatedAt': datetime.datetime(2022, 6, 18, 4, 39, 53, 121000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '65741481-4293-425e-9a01-79eb5c7ec0c0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '65741481-4293-425e-9a01-79eb5

,pid,btrim,starttime,substring


In [ ]:
pid = 'pid'
redshift_s3_connect.cancel_sql_command(pid)